In [1]:
# Imports

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


import numpy as np
import os
import pandas as pd
import re
import string
import regex

import spacy
from spacy import displacy

import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import time

C:\Users\Armad\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\Armad\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#s1 ---Preprocess/clean all origanal .csv's and store them in one .csv [Start]:

In [3]:
# Read Datasets
data_location = '.\\data_original\\' #Change path to where original .csv's are
my_directory = os.listdir(data_location) 
print(my_directory)

['1864.csv', '1900.csv']


In [4]:
# ==== Data Preprocessing/Cleaning: 

In [11]:
# --- Preprocess/clean original data:

fields = ['article', 'title']

# Dictionary of nltk's stopwords
stop = set(stopwords.words('english'))

start_time_total = time.time()

for year_ in my_directory:
    chunksize = 100000
    j = 1
    print("Processing: ", str(year_))
    start_time = time.time()
    for df in pd.read_csv(data_location + year_, chunksize=chunksize, iterator=True, skipinitialspace=True, usecols=fields):
        
        #remove spaces in "column names":
        df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
        
        print("Data preprocessing --> Removing URL's ...")
        df['article'] = df['article'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])# Data preprocessing --> remove URL 
        df['title'] = df['title'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])# Data preprocessing --> remove URL 
        
        print("Data preprocessing --> Convert all to lower case ...")
        df = df.astype(str).apply(lambda x: x.str.lower()) # Data preprocessing --> all lower case
        
        print("Data preprocessing --> Removing Punctuation ...")
        df = df.astype(str).apply(lambda x: x.str.translate(str.maketrans('', '', string.punctuation))) # Data preprocessing --> remove Punctuation  
        
        print("Data preprocessing --> Removing Numbers ...")
        df = df.astype(str).apply(lambda x: x.str.translate(str.maketrans('', '', string.digits))) # Data preprocessing --> remove numbers          
        
        df.index += j # Table Index 
        
        #------Remove "digitized text article available":
        #digitized text article available,gen bullers reports,1900
        #Digitized text of this article is not available
        df['article'] = df['article'].apply(lambda x: regex.sub('digitized text of this article is not available', '', str(x)))
        #------
        
        ## Remove non-latin characters (1)
        print("Data preprocessing --> Removing non-latin characters ...")
        #df['article'] = df['article'].apply(lambda x: regex.sub(r'[^\p{Latin}]', u'', str(x)))# Data preprocessing --> remove non-latin characters (1)
        df['article'] = df['article'].apply(lambda x: regex.sub(r'[^\p{Latin}{/^\s*$/}]', u'', str(x)))# Data preprocessing --> remove non-latin characters (1)
        df['title'] = df['title'].apply(lambda x: regex.sub(r'[^\p{Latin}{/^\s*$/}]', u'', str(x)))# Data preprocessing --> remove non-latin characters (1)
        ## Remove non-latin characters (2)
        df['article'] = df['article'].apply(lambda x:  re.sub(r'[^\x00-\x7F\x80-\xFF\u0100-\u017F\u0180-\u024F\u1E00-\u1EFF]', u'', str(x))  )# Data preprocessing --> remove non-latin characters (2)
        df['title'] = df['title'].apply(lambda x:  re.sub(r'[^\x00-\x7F\x80-\xFF\u0100-\u017F\u0180-\u024F\u1E00-\u1EFF]', u'', str(x))  )# Data preprocessing --> remove non-latin characters (2)
  
        # Remove StopWords: [nltk]
        print("Data preprocessing --> Removing StopWords ...")
        #from nltk import word_tokenize
        df['title'] = df['title'].apply(lambda x: [item for item in word_tokenize(x) if item not in stop])
        df['article'] = df['article'].apply(lambda x: [item for item in word_tokenize(x) if item not in stop])
        
        #Stemming: [OPTIOANL]
        print("Data preprocessing --> Stemming ...")
        ##stemming = PorterStemmer()
        ##df['article'] = df['article'].apply(lambda x: [stemming.stem(item) for item in x]) # Data preprocessing --> Stemming  ##// O4en' G
        
        # Rejoin words after Tokenization  [nltk]
        print("Re-join words after Tokenization ...")
        df['title'] = df['title'].apply(lambda x:( " ".join(x)))
        df['article'] = df['article'].apply(lambda x:( " ".join(x)))      
        
        df['year'] = year_.replace('.csv', '')        
        title_ = year_.replace('.csv', '')        
        df = df[['year', 'article', 'title']]

        
        #"" save to csv
        print("Saving: ", title_, ".csv")
        path_cleaned =  '.\\data_cleaned\\'
        df.to_csv(path_cleaned + '_cleaned_'+title_+'.csv', index = None)#, header = False)        
        
        j = df.index[-1] + 1 # Table Index 
        #print("j2 : ", j)
        print("Done! --> Rows processed: ", j)
    end_time = time.time()
    print("Completed: ", str(title_) , "--> Time: ", end_time-start_time )
    print("="*10)
    
end_time_total = time.time()
print("*"*10)
print("Data Preprocessing/Cleaning Done! ", " --> Total Time: ", end_time_total-start_time_total )

<>:21: DeprecationWarning: invalid escape sequence \/
<>:22: DeprecationWarning: invalid escape sequence \/
<>:21: DeprecationWarning: invalid escape sequence \/
<>:22: DeprecationWarning: invalid escape sequence \/
<>:21: DeprecationWarning: invalid escape sequence \/
<>:22: DeprecationWarning: invalid escape sequence \/
<ipython-input-11-2e0b68a8f650>:21: DeprecationWarning: invalid escape sequence \/
  df['article'] = df['article'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])# Data preprocessing --> remove URL
<ipython-input-11-2e0b68a8f650>:22: DeprecationWarning: invalid escape sequence \/
  df['title'] = df['title'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])# Data preprocessing --> remove URL


Processing:  1864.csv
Data preprocessing --> Removing URL's ...
Data preprocessing --> Convert all to lower case ...
Data preprocessing --> Removing Punctuation ...
Data preprocessing --> Removing Numbers ...
Data preprocessing --> Removing non-latin characters ...
Data preprocessing --> Removing StopWords ...
Data preprocessing --> Stemming ...
Re-join words after Tokenization ...
Saving:  1864 .csv
Done! --> Rows processed:  15829
Completed:  1864 --> Time:  70.89200234413147
Processing:  1900.csv
Data preprocessing --> Removing URL's ...
Data preprocessing --> Convert all to lower case ...
Data preprocessing --> Removing Punctuation ...
Data preprocessing --> Removing Numbers ...
Data preprocessing --> Removing non-latin characters ...
Data preprocessing --> Removing StopWords ...
Data preprocessing --> Stemming ...
Re-join words after Tokenization ...
Saving:  1900 .csv
Done! --> Rows processed:  69609
Completed:  1900 --> Time:  31.34299898147583
**********
Data Preprocessing/Clea